# Magentic オーケストレーション：マルチデータベース対応エージェントシステム

## 概要

このノートブックでは、Semantic Kernel の Magentic オーケストレーション機能を活用して、複数種類のデータベースに対応する高度なマルチエージェントシステムを構築する方法を学習します。

### 主な学習内容
- MagenticOrchestration による自動的な制御フロー
- StandardMagenticManager による高度な会話管理
- 異種データベース間での統合クエリ処理
- 専門化されたエージェントによる役割分担システム

### 本章で作る構成
- マルチデータベース統合システム
    - PostgresGenerationAgent：PostgreSQL用SQLクエリ生成
    - PostgresExecutionAgent：PostgreSQL用SQLクエリ実行
    - CosmosGenerationAgent：CosmosDB NoSQL用SQLクエリ生成
    - CosmosExecutionAgent：CosmosDB NoSQL用SQLクエリ実行
    - StandardMagenticManager：4つのエージェント間の自動協調制御

### 実用的な活用場面

- 異種データベース統合分析システム
- ハイブリッドクラウドデータ基盤の統一アクセス
- リレーショナルデータとNoSQLデータの横断検索
- 企業データレイクの統合クエリインターフェース
- マルチベンダーデータベース環境の統一管理

# ライブラリのインポート

Magentic オーケストレーションによるマルチデータベースシステムの構築に必要なライブラリをインポートします。

## 主要コンポーネント

- **MagenticOrchestration**: 高度な自動制御フローによる複数エージェント管理
- **StandardMagenticManager**: 構造化出力をサポートする会話マネージャー
- **psycopg2**: PostgreSQLデータベース接続
- **azure.cosmos**: CosmosDB NoSQL接続
- **DefaultAzureCredential**: Azure認証の統合管理

In [ ]:
import os
import json
import asyncio
import datetime
import psycopg2

from dotenv import load_dotenv, find_dotenv
from typing import Annotated, Any, List, Optional
from pydantic import BaseModel

from azure.identity import DefaultAzureCredential
from azure.cosmos import CosmosClient
from azure.cosmos.exceptions import CosmosHttpResponseError

from semantic_kernel.agents import (
    Agent, ChatCompletionAgent, GroupChatOrchestration, 
    GroupChatManager, BooleanResult, StringResult, MessageResult
)
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureChatPromptExecutionSettings
from semantic_kernel.contents import (
    ChatMessageContent, FunctionCallContent, FunctionResultContent, 
    AuthorRole, TextContent, ChatHistory
)
from semantic_kernel.functions import kernel_function
from semantic_kernel.functions.kernel_arguments import KernelArguments
from semantic_kernel.agents import (
    MagenticOrchestration,
    StandardMagenticManager,
)

# 環境変数の設定

マルチデータベース対応システムに必要な環境変数を取得します。

## 必要な設定値

- **Azure OpenAI関連**: APIエンドポイント、デプロイメント名、認証キー
- **PostgreSQL関連**: リレーショナルデータベース接続情報
- **CosmosDB関連**: NoSQLデータベース接続情報（エンドポイント、キー、データベース、コンテナ）
- **Azure認証関連**: プロジェクトエンドポイントとAzure統合認証

In [17]:
load_dotenv(override=True)

# Azure OpenAI 設定
PROJECT_ENDPOINT = os.getenv("PROJECT_ENDPOINT")
AZURE_DEPLOYMENT_NAME = os.getenv("AZURE_DEPLOYMENT_NAME")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")

# PostgreSQL 設定
PG_HOST = os.getenv("PG_HOST")
PG_PORT = os.getenv("PG_PORT", "5432")
PG_DB = os.getenv("PG_DB")
PG_USER = os.getenv("PG_USER")
PG_PASS = os.getenv("PG_PASS")

# CosmosDB 設定
COSMOS_ENDPOINT = os.getenv("COSMOS_ENDPOINT")
COSMOS_KEY = os.getenv("COSMOS_KEY")
COSMOS_DB = os.getenv("COSMOS_DB")
COSMOS_CONTAINER = os.getenv("COSMOS_CONTAINER")

# データベース接続設定

PostgreSQLとCosmosDBの両方のデータベース接続設定を構成します。

## 接続設定の特徴
- **PostgreSQL**: psycopg2ライブラリを使用した従来のリレーショナルデータベース接続
- **CosmosDB**: Azure Cosmos ClientによるNoSQLデータベース接続

In [18]:
# PostgreSQL 接続設定
PG_CONFIG = {
    "user": PG_USER,
    "password": PG_PASS,
    "dbname": PG_DB,
    "host": PG_HOST,
    "port": int(PG_PORT),
}

# CosmosDB 接続設定
COSMOS_CONFIG = {
    "endpoint": COSMOS_ENDPOINT,
    "key": COSMOS_KEY,
    "database": COSMOS_DB,
    "container": COSMOS_CONTAINER
}

print("データベース接続設定が完了しました。")

データベース接続設定が完了しました。


# プラグインクラスの定義

PostgreSQLとCosmosDBの両方に対応する専門化されたプラグインクラスを定義します。

## アーキテクチャ設計
- **責任分離**: スキーマ取得とクエリ実行を分離
- **データベース別実装**: PostgreSQLとCosmosDBそれぞれに最適化
- **エラーハンドリング**: 各データベース固有のエラー処理
- **JSON統一出力**: 一貫した結果フォーマット

## PostgreSQL スキーマプラグイン

PostgreSQLデータベースの構造情報を取得する専門プラグインです。

### 機能詳細
- **get_tables**: 公開スキーマ内の全テーブル一覧を取得
- **get_table_schema**: 指定テーブルのカラム情報（名前・データ型）を取得

### PostgreSQL固有の特徴
- information_schemaを活用したメタデータ取得
- psycopg2による安全な接続管理
- リレーショナルデータベースの正規化構造に対応

In [19]:
class PostgresSchemaPlugin:
    """
    PostgreSQLのテーブル一覧・スキーマ取得専用プラグイン
    """
    def __init__(self):
        self.pg_config = PG_CONFIG

    def _get_connection(self):
        return psycopg2.connect(**self.pg_config)

    @kernel_function(
        name="get_tables",
        description="PostgreSQLデータベース内のテーブル一覧をJSON文字列で取得します。"
    )
    def get_tables(
        self,
    ) -> Annotated[str, "テーブル一覧を含むJSON文字列（例: {'tables': ['table1', 'table2']}）"]:
        with self._get_connection() as conn, conn.cursor() as cur:
            cur.execute("SELECT tablename FROM pg_tables WHERE schemaname = 'public';")
            tables = [row[0] for row in cur.fetchall()]
        return json.dumps({"tables": tables})

    @kernel_function(
        name="get_table_schema",
        description="指定したテーブルのスキーマ情報（カラム名、型など）をJSON文字列で取得します。"
    )
    def get_table_schema(
        self,
        table_name: Annotated[str, "スキーマ情報を取得したいテーブル名"]
    ) -> Annotated[str, "カラム情報を含むJSON文字列（例: {'columns': [{'name': 'id', 'type': 'integer'}, ...]}）"]:
        with self._get_connection() as conn, conn.cursor() as cur:
            cur.execute("""
                SELECT column_name, data_type
                FROM information_schema.columns
                WHERE table_name = %s;
            """, (table_name,))
            columns = [{"name": row[0], "type": row[1]} for row in cur.fetchall()]
        return json.dumps({"columns": columns})

print("PostgreSQLスキーマプラグインが定義されました。")

PostgreSQLスキーマプラグインが定義されました。


## PostgreSQL クエリ実行プラグイン

PostgreSQL用SQLクエリを実行し、結果を取得する専門プラグインです。

### 実行機能
- **execute_sql**: 標準SQLクエリの実行と結果取得
- **カーソル管理**: psycopg2による効率的なデータ取得
- **結果変換**: 辞書形式への自動変換とJSON出力

### 安全性対策
- トランザクション管理による安全な実行
- SQLインジェクション対策
- 詳細なエラーログとエラーハンドリング

In [20]:
class PostgresQueryPlugin:
    """
    PostgreSQLのSQL実行専用プラグイン
    """
    def __init__(self):
        self.pg_config = PG_CONFIG

    def _get_connection(self):
        return psycopg2.connect(**self.pg_config)

    @kernel_function(
        name="execute_sql",
        description="任意のSQL文を実行し、結果をJSON文字列で返します。（SELECTのみ対応を推奨）"
    )
    def execute_sql(
        self,
        sql: Annotated[str, "実行したいSQLクエリ（例: 'SELECT * FROM users'）"]
    ) -> Annotated[str, "クエリ結果のJSON文字列（例: {'rows': [...]}）"]:
        try:
            print(f"Executing PostgreSQL SQL: {sql}")
            with self._get_connection() as conn, conn.cursor() as cur:
                cur.execute(sql)
                columns = [desc[0] for desc in cur.description] if cur.description else []
                rows = [dict(zip(columns, row)) for row in cur.fetchall()] if columns else []
            print(f"PostgreSQL SQL executed successfully. Rows returned: {len(rows)}")
            return json.dumps({"rows": rows, "row_count": len(rows)})
        except Exception as e:
            error_msg = str(e)
            print(f"PostgreSQL SQL execution failed: {error_msg}")
            raise Exception(error_msg)

print("PostgreSQLクエリプラグインが定義されました。")

PostgreSQLクエリプラグインが定義されました。


## CosmosDB スキーマプラグイン

CosmosDB NoSQLデータベースのドキュメント構造を分析する専門プラグインです。

### 機能詳細
- **get_container_info**: コンテナの基本情報を取得
- **get_sample_documents**: サンプルドキュメントからスキーマ構造を推定
- **get_partition_key_path**: パーティションキー設定を取得

### NoSQL固有の特徴
- スキーマレス構造への対応
- ドキュメント階層の動的分析
- パーティションキーとインデックスポリシーの取得
- 柔軟なフィールド構造の理解

In [21]:
class CosmosSchemaPlugin:
    """
    CosmosDB NoSQLのコンテナー情報取得専用プラグイン
    """
    def __init__(self):
        self.endpoint = COSMOS_CONFIG["endpoint"]
        self.key = COSMOS_CONFIG["key"]
        self.database_name = COSMOS_CONFIG["database"]
        self.container_name = COSMOS_CONFIG["container"]
        self.client = CosmosClient(self.endpoint, self.key)
        self.database = self.client.get_database_client(self.database_name)
        self.container = self.database.get_container_client(self.container_name)

    @kernel_function(
        name="get_container_info",
        description="CosmosDBコンテナーの基本情報をJSON文字列で取得します。"
    )
    def get_container_info(
        self,
    ) -> Annotated[str, "コンテナー情報を含むJSON文字列（例: {'container': 'container_name', 'database': 'db_name'}）"]:
        try:
            container_properties = self.container.read()
            info = {
                "container": self.container_name,
                "database": self.database_name,
                "partition_key": container_properties.get("partitionKey", {}),
                "indexing_policy": container_properties.get("indexingPolicy", {})
            }
            return json.dumps(info, ensure_ascii=False)
        except Exception as e:
            error_msg = f"CosmosDB container info retrieval failed: {str(e)}"
            print(error_msg)
            raise Exception(error_msg)

    @kernel_function(
        name="get_sample_documents",
        description="CosmosDBコンテナーからサンプルドキュメントを取得し、スキーマ構造を把握します。"
    )
    def get_sample_documents(
        self,
        limit: Annotated[int, "取得するサンプルドキュメント数（デフォルト: 5）"] = 5
    ) -> Annotated[str, "サンプルドキュメントを含むJSON文字列"]:
        try:
            print(f"Fetching {limit} sample documents from CosmosDB")
            query = f"SELECT TOP {limit} * FROM c"
            query_iter = self.container.query_items(
                query=query,
                enable_cross_partition_query=True
            )
            samples = [dict(item) for item in query_iter]
            print(f"CosmosDB sample documents fetched successfully. Count: {len(samples)}")
            return json.dumps({"samples": samples, "count": len(samples)}, ensure_ascii=False)
        except Exception as e:
            error_msg = f"CosmosDB sample document retrieval failed: {str(e)}"
            print(error_msg)
            raise Exception(error_msg)

    @kernel_function(
        name="get_partition_key_path",
        description="CosmosDBコンテナーのパーティションキー（path）を取得します。"
    )
    def get_partition_key_path(
        self,
    ) -> Annotated[str, "パーティションキーのパス（例: '/tweet_id'）"]:
        """
        CosmosDBコンテナーのパーティションキー（path）をJSONで返します。
        """
        try:
            container_properties = self.container.read()
            pk = container_properties.get("partitionKey", {})
            pk_paths = pk.get("paths", [])
            # CosmosDBのパーティションキーは配列なので、通常は1個（複数指定も将来あるかも）
            if pk_paths:
                return json.dumps({"partition_key_path": pk_paths[0]}, ensure_ascii=False)
            else:
                return json.dumps({"partition_key_path": None}, ensure_ascii=False)
        except Exception as e:
            error_msg = f"CosmosDB partition key retrieval failed: {str(e)}"
            print(error_msg)
            raise Exception(error_msg)


print("CosmosDBスキーマプラグインが定義されました。")

CosmosDBスキーマプラグインが定義されました。


## CosmosDB クエリ実行プラグイン

CosmosDB NoSQL用SQLクエリを実行し、結果を取得する専門プラグインです。

### 実行機能
- **execute_cosmos_sql**: CosmosDB SQL API によるクエリ実行
- **クロスパーティション対応**: 複数パーティションにまたがるクエリ実行
- **ドキュメント取得**: JSON ドキュメントの直接取得と変換

### CosmosDB特有の対応
- CosmosHttpResponseError の専用エラーハンドリング
- enable_cross_partition_query による柔軟なクエリ実行
- NoSQL特有のクエリ構文への対応

In [22]:
class CosmosQueryPlugin:
    """
    CosmosDB NoSQL SQLクエリ実行専用プラグイン
    """
    def __init__(self):
        self.endpoint = COSMOS_CONFIG["endpoint"]
        self.key = COSMOS_CONFIG["key"]
        self.database_name = COSMOS_CONFIG["database"]
        self.container_name = COSMOS_CONFIG["container"]
        self.client = CosmosClient(self.endpoint, self.key)
        self.database = self.client.get_database_client(self.database_name)
        self.container = self.database.get_container_client(self.container_name)

    @kernel_function(
        name="execute_cosmos_sql",
        description="CosmosDBにSQLクエリ（SELECT系）を実行し、結果をJSON文字列で返します"
    )
    def execute_cosmos_sql(
        self,
        sql: Annotated[str, "CosmosDBのSQLクエリ（例: 'SELECT * FROM c WHERE c.status=\"active\"' ）"]
    ) -> Annotated[str, "クエリ結果のJSON文字列（例: {'rows': [...]} ）"]:
        try:
            print(f"Executing CosmosDB SQL: {sql}")
            query_iter = self.container.query_items(
                query=sql,
                enable_cross_partition_query=True
            )
            rows = [dict(item) for item in query_iter]
            print(f"CosmosDB SQL executed successfully. Rows returned: {len(rows)}")
            return json.dumps({"rows": rows, "row_count": len(rows)}, ensure_ascii=False)
        except CosmosHttpResponseError as e:
            error_msg = f"CosmosDB error: {str(e)}"
            print(error_msg)
            raise Exception(error_msg)
        except Exception as e:
            error_msg = f"CosmosDB query execution failed: {str(e)}"
            print(error_msg)
            raise Exception(error_msg)

print("CosmosDBクエリプラグインが定義されました。")

CosmosDBクエリプラグインが定義されました。


# エージェントの作成と設定

マルチデータベース対応システムの基盤となるサービスとエージェントを作成します。

## システム構成要素
- **Azure OpenAI サービス**: 全エージェントの言語理解基盤
- **構造化出力設定**: 一貫した応答フォーマットの制御
- **専門化エージェント**: データベース別の特化機能

## 設計思想
各データベースに対して生成と実行の2つのエージェントを配置し、責任を明確に分離

In [23]:
# Azure OpenAI サービスの初期化
azure_completion_service = AzureChatCompletion(
    service_id="azure_completion_agent",
    deployment_name=AZURE_DEPLOYMENT_NAME,
    endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY
)

# 構造化出力の設定
settings = AzureChatPromptExecutionSettings()
response_format_dict = {
    "type": "json_schema",
    "json_schema": {
        "name": "QueryExecutionResult",
        "schema": {
            "type": "object",
            "properties": {
                "status": {
                    "type": "string",
                    "enum": ["success", "error"]
                },
                "result": {
                    "type": "string",
                    "description": "JSON string containing the query execution result or error message"
                }
            },
            "required": ["status", "result"],
            "additionalProperties": False
        },
        "strict": True
    }
}
settings.response_format = response_format_dict

print("Azure OpenAIサービスと構造化出力設定が完了しました。")

Azure OpenAIサービスと構造化出力設定が完了しました。


## エージェントの定義

PostgreSQLに特化した2つのエージェントと、CosmosDBに特化した2つのエージェントを作成します。

### PostgresSQLGenerationAgent
- **役割**: 自然言語からPostgreSQL用SQLクエリの生成
- **特徴**: リレーショナルデータベース構造の理解と最適化
- **アプローチ**: スキーマ分析→クエリ生成→検証の段階的処理

### PostgresSQLExecutionAgent
- **役割**: 生成されたSQLクエリの安全な実行
- **特徴**: 構造化出力による制御された結果返却
- **安全性**: SQLインジェクション対策と実行ログ

### CosmosDBSQLGenerationAgent
- **役割**: 自然言語からCosmosDB NoSQL用SQLクエリの生成
- **特徴**: スキーマレス構造とドキュメント階層の理解
- **アプローチ**: サンプル分析→構造理解→NoSQL特化クエリ生成

### CosmosDBSQLExecutionAgent
- **役割**: CosmosDB専用SQLクエリの実行と結果処理
- **特徴**: パーティション横断クエリとJSON直接処理
- **最適化**: CosmosDB固有のエラーハンドリングと性能考慮

In [25]:
async def create_database_agents():
    """
    PostgreSQLとCosmosDBの4体のエージェントを作成する関数
    """
    
    # PostgreSQL クエリ生成エージェント
    postgres_query_generation_agent = ChatCompletionAgent(
        name="PostgresGenerationAgent",  # 名前を短縮・明確化
        description="ユーザーの自然言語質問からPostgreSQL用のSQLクエリを生成するエージェント",
        instructions=(
            "あなたはユーザーの質問に基づいて、PostgreSQLデータベースで実行可能なSQLクエリ（SELECT文のみ）を生成する役割です。\n"
            "【必ず守ること】\n"
            "1. SQLクエリを生成する前に、まずデータベース内のテーブル一覧を取得してください。\n"
            "2. 次に、関連しそうなテーブルごとにカラム情報も取得し、スキーマ構成を十分に把握してください。\n"
            "3. 取得したテーブル・カラム情報（スキーマ情報）を必ず参照したうえで、PostgreSQL用の正しいSQLを生成してください。\n"
            "4. ユーザーの質問がPostgreSQLに関連する場合のみ、SQLクエリを生成してください。\n"
            "5. CosmosDBに関する質問の場合は、他のエージェントに任せてください。\n"
            "6. 最終的に正しいSQLが完成したら、そのSQL文のみを返してください。\n"
            "【注意事項】\n"
            "・SQLインジェクションや危険なクエリ生成は厳禁です。\n"
            "・不要なコメントや説明文は出力せず、SQL文のみを返してください。\n"
            "・必ず実際のテーブル名とカラム名を確認してからSQLを生成してください。"
        ),
        service=azure_completion_service,
        plugins=[PostgresSchemaPlugin()]
    )

    # PostgreSQL クエリ実行エージェント
    postgres_query_execution_agent = ChatCompletionAgent(
        service=azure_completion_service,
        name="PostgresExecutionAgent",  # 名前を短縮・明確化
        description="PostgreSQLのSQL文を実行し、結果を返すエージェント",
        instructions=(
            "あなたは与えられたSQLクエリを、必ずそのままPostgreSQLで実行する役割です。\n"
            "【実行手順】\n"
            "1. PostgreSQL用のSQLクエリが提供された場合、execute_sql関数を使用して実行してください。\n"
            "2. execute_sql関数の結果をそのまま返してください。\n"
            "3. CosmosDBに関するクエリの場合は、他のエージェントに任せてください。\n"
            "【重要】\n"
            "・必ずexecute_sql関数を呼び出してSQLを実行してください。\n"
            "・危険なSQL（データ破壊やセキュリティリスクを伴うもの）は実行しないでください。\n"
            "・結果を分かりやすく表示してください。"
        ),
        plugins=[PostgresQueryPlugin()]
    )

    # CosmosDB クエリ生成エージェント
    cosmos_query_generation_agent = ChatCompletionAgent(
        name="CosmosGenerationAgent",  # 名前を短縮・明確化
        description="ユーザーの自然言語質問からCosmosDB NoSQL用のSQLクエリを生成するエージェント",
        instructions=(
            "あなたはユーザーの質問に基づいて、CosmosDB NoSQLで実行可能なSQLクエリ（SELECT文のみ）を生成する役割です。\n"
            "【必ず守ること】\n"
            "1. SQLクエリを生成する前に、まずCosmosDBコンテナーの基本情報を取得してください。\n"
            "2. 次に、サンプルドキュメントを取得し、ドキュメント構造とフィールドを十分に把握してください。\n"
            "3. 取得したコンテナー情報とサンプルドキュメント構造を必ず参照したうえで、CosmosDB NoSQL用の正しいSQLを生成してください。\n"
            "4. CosmosDB NoSQLでは、'c'をコンテナーのエイリアスとして使用し、フィールドアクセスは'c.fieldname'形式で行ってください。\n"
            "5. ユーザーの質問がCosmosDBに関連する場合のみ、SQLクエリを生成してください。\n"
            "6. PostgreSQLに関する質問の場合は、他のエージェントに任せてください。\n"
            "7. 最終的に正しいSQLが完成したら、そのSQL文のみを返してください。\n"
            "【CosmosDB NoSQL特有の注意事項】\n"
            "・文字列の比較では二重引用符を使用してください（例: c.status = \"active\"）\n"
            "・クロスパーティションクエリが必要な場合があることを理解してください\n"
            "・必ず実際のフィールド名を確認してからSQLを生成してください\n"
            "・NoSQLドキュメントの階層構造に注意してアクセスしてください（例: c.address.city）"
        ),
        service=azure_completion_service,
        plugins=[CosmosSchemaPlugin()]
    )

    # CosmosDB クエリ実行エージェント
    cosmos_query_execution_agent = ChatCompletionAgent(
        service=azure_completion_service,
        name="CosmosExecutionAgent",  # 名前を短縮・明確化
        description="CosmosDB NoSQLのSQL文を実行し、結果を返すエージェント",
        instructions=(
            "あなたは与えられたSQLクエリを、必ずそのままCosmosDB NoSQLで実行する役割です。\n"
            "【実行手順】\n"
            "1. CosmosDB NoSQL用のSQLクエリが提供された場合、execute_cosmos_sql関数を使用して実行してください。\n"
            "2. execute_cosmos_sql関数の結果をそのまま返してください。\n"
            "3. PostgreSQLに関するクエリの場合は、他のエージェントに任せてください。\n"
            "【重要】\n"
            "・必ずexecute_cosmos_sql関数を呼び出してSQLを実行してください。\n"
            "・危険なクエリ（データ破壊やセキュリティリスクを伴うもの）は実行しないでください。\n"
            "・結果を分かりやすく表示してください。"
        ),
        plugins=[CosmosQueryPlugin()]
    )

    return [
        postgres_query_generation_agent,
        postgres_query_execution_agent,
        cosmos_query_generation_agent,
        cosmos_query_execution_agent
    ]

print("データベースエージェント作成関数が定義されました。")

データベースエージェント作成関数が定義されました。


# Magentic オーケストレーション実行

Magentic オーケストレーションによるマルチデータベースクエリシステムを実行します。

## 実行フロー
1. **エージェント群の初期化**: 4つの専門化エージェントの作成
2. **StandardMagenticManager**: 自動制御マネージャーの設定
3. **MagenticOrchestration**: 全体統合システムの構築
4. **自動実行**: ユーザー質問に基づく適切なデータベース選択と処理
5. **結果統合**: 複数データベースの結果を統合した最終回答

## 高度な機能
- **コールバック監視**: エージェント間の通信とファンクション呼び出しの可視化
- **エラー回復**: データベース固有エラーからの自動回復
- **最適化**: 質問内容に基づく効率的なデータベース選択
- **自律制御**: AI による動的なエージェント選択と実行計画

In [29]:
# エージェントのレスポンスを処理するコールバック関数
def agent_response_callback(message: ChatMessageContent) -> None:
    """エージェントの応答とファンクション呼び出しを監視するコールバック"""
    print(f"{message.name}: {message.content}")
    for item in message.items:
        if isinstance(item, FunctionCallContent):
            print(f"関数呼び出し: '{item.name}' with arguments '{item.arguments}'")
        if isinstance(item, FunctionResultContent):
            print(f"関数結果: '{item.name}' returned '{item.result}'")

async def run_magentic_database_orchestration(task: str):
    """
    MagenticOrchestrationを使用してマルチデータベースクエリを実行する関数
    
    Args:
        task (str): ユーザーのタスク（自然言語での質問）
    """
    
    # 1. magenticオーケストレーションを4体のエージェントとmagenticマネージャーで作成
    # StandardMagenticManagerは構造化出力をサポートするチャット補完モデルが必要
    magentic_orchestration = MagenticOrchestration(
        members=await create_database_agents(),
        manager=StandardMagenticManager(chat_completion_service=azure_completion_service),
        # agent_response_callback=agent_response_callback,
    )

    # 2. ランタイムを作成して開始
    runtime = InProcessRuntime()
    runtime.start()

    # 3. タスクとランタイムでオーケストレーションを呼び出し
    print(f"Starting Magentic Orchestration for task: {task}")
    print("=" * 100)
    
    orchestration_result = await magentic_orchestration.invoke(
        task=task,
        runtime=runtime,
    )

    # 4. 結果を待機
    final_result = await orchestration_result.get()

    print("=" * 100)
    print(f"Final Result:")
    print(f"{final_result}")

    # 5. アイドル時にランタイムを停止
    await runtime.stop_when_idle()
    
    return final_result

print("magenticオーケストレーション実行関数が定義されました。")

magenticオーケストレーション実行関数が定義されました。


# PostgreSQL クエリの実行例

PostgreSQLデータベースに対する複雑な集計クエリの実行例です。

In [27]:
# PostgreSQLデータベースに対するクエリの実行例
analysis_task = "2024年11月の月別で最も販売数が多かった商品は？"

print("Magenticオーケストレーションを開始します...")
await run_magentic_database_orchestration(analysis_task)

Magenticオーケストレーションを開始します...
Starting Magentic Orchestration for task: 2024年11月の月別で最も販売数が多かった商品は？
Starting Magentic Orchestration for task: 2024年11月の月別で最も販売数が多かった商品は？
PostgresGenerationAgent: 
関数呼び出し: 'PostgresSchemaPlugin-get_tables' with arguments '{}'
PostgresGenerationAgent: 
関数結果: 'PostgresSchemaPlugin-get_tables' returned '{"tables": ["inventory", "order_details", "products", "users", "orders", "categories", "prefectures"]}'
PostgresGenerationAgent: 
関数呼び出し: 'PostgresSchemaPlugin-get_tables' with arguments '{}'
PostgresGenerationAgent: 
関数結果: 'PostgresSchemaPlugin-get_tables' returned '{"tables": ["inventory", "order_details", "products", "users", "orders", "categories", "prefectures"]}'
PostgresGenerationAgent: 
関数呼び出し: 'PostgresSchemaPlugin-get_table_schema' with arguments '{"table_name": "order_details"}'
関数呼び出し: 'PostgresSchemaPlugin-get_table_schema' with arguments '{"table_name": "orders"}'
関数呼び出し: 'PostgresSchemaPlugin-get_table_schema' with arguments '{"table_name": "produ

ChatMessageContent(inner_content=ChatCompletion(id='chatcmpl-BynTQ8JnajMUbBojIbWv4gmc2FNhW', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='2024年11月に最も販売数が多かった商品は「Xbox Game Pass」で、合計販売数は82個でした。', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1753829556, model='gpt-4.1-2025-04-14', object='chat.completion', service_tier=None, system_fingerprint='fp_b663f05c2c', usage=CompletionUsage(completion_tokens=31, prompt_tokens=1094, total_tokens=1125, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, c

## CosmosDB も交えた複雑なクエリの実行例

PostgreSQLだけでなく、CosmosDB NoSQLデータベースに対してもクエリを実行しないと回答を得られない質問です。

In [28]:
# PostgreSQL と CosmosDB データベース両方に対するクエリの実行例
analysis_task = "Xbox Game Passは2024年月別で最も売れた月は何月？またレビューデータも用いて売れた要因を分析して。"

print("Magenticオーケストレーションを開始します...")
await run_magentic_database_orchestration(analysis_task)

Magenticオーケストレーションを開始します...
Starting Magentic Orchestration for task: Xbox Game Passは2024年月別で最も売れた月は何月？またレビューデータも用いて売れた要因を分析して。
Starting Magentic Orchestration for task: Xbox Game Passは2024年月別で最も売れた月は何月？またレビューデータも用いて売れた要因を分析して。
PostgresGenerationAgent: 
関数呼び出し: 'PostgresSchemaPlugin-get_tables' with arguments '{}'
PostgresGenerationAgent: 
関数結果: 'PostgresSchemaPlugin-get_tables' returned '{"tables": ["inventory", "order_details", "products", "users", "orders", "categories", "prefectures"]}'
PostgresGenerationAgent: 
関数呼び出し: 'PostgresSchemaPlugin-get_tables' with arguments '{}'
PostgresGenerationAgent: 
関数結果: 'PostgresSchemaPlugin-get_tables' returned '{"tables": ["inventory", "order_details", "products", "users", "orders", "categories", "prefectures"]}'
PostgresGenerationAgent: 
関数呼び出し: 'PostgresSchemaPlugin-get_table_schema' with arguments '{"table_name": "orders"}'
関数呼び出し: 'PostgresSchemaPlugin-get_table_schema' with arguments '{"table_name": "order_details"}'
関数呼び出し: 'PostgresSchema

ChatMessageContent(inner_content=ChatCompletion(id='chatcmpl-BynUZoPoWznvYccEOWLXbdFnwyNvY', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='ご質問ありがとうございます。  \n2024年のXbox Game Passに関して、月別の売上データ自体は今回のデータベース内では確認できませんでしたが、レビュー情報から当時の状況を詳細に分析しました。\n\n**【結論：最も盛り上がった月】**  \n2024年のレビュー内容や投稿数・内容の集中度から、“**4月**”が特に注目された月となっています。  \n4月15日を中心に「キャンペーン」「セール」「特典」が実施されたとのコメントや高評価（ほとんどが5段階で5）が多く、4月に加入や利用が大きく伸びたことが推測されます。\n\n**【売上増加・高評価の要因分析】**  \n- 4月には「特典つきキャンペーン」「限定セール」「割引イベント」などのプロモーションが集中し、多くのユーザーから高評価のレビューが投稿されています。\n- レビューから、特典や割引が「加入のきっかけ」「満足度向上」につながったことが読み取れます。\n- 上記以外の月（6月、7月、10月、11月など）でも好意的レビューは見られるものの、4月の投稿数・内容の熱量には及びません。\n- 特に「初月無料」や「抽選ギフト」など、ユーザーが目を引くプロモーション施策が売上・加入増の原動力になったと考えられます。\n\n**【まとめ】**  \nレビュー分析を通じて、2024年は「4月」が最も盛り上がり、キャンペーンやセール、特典施策が売上を伸ばした主要因であることが分かりました。  \nもしさらに詳細な月別データや、特定キャンペーンの影響度分析などご希望がありましたらお知らせください。', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None), content_filter_resu